## Historic data analyser
This Notebook aims to manipulate input historic data, to determine returns statistical values

In [2]:
import pandas as pd
from sqlalchemy import create_engine, Table, select, MetaData, inspect
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import date, timedelta

# Reuse Get_data function again to query database
def Get_data(ticker):
    engine = create_engine('sqlite:///Data.db')
    metadata = MetaData()
    metadata.reflect(bind=engine)
    Session = sessionmaker(bind=engine)
    session = Session()

    if ticker in metadata.tables:
        table = Table(ticker, metadata, autoload_with=engine)
        query = select(table).limit(10000)
        result = session.execute(query)
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        session.close()
        return df
    else:
        print(f"Table '{ticker}' not found in the database.")
        session.close()
        return pd.DataFrame()

    
name = input(f"Enter ticker:")
df = Get_data("GOOG")

df["daily_returns"] = df["adj_close"].pct_change() 

Enter ticker:GOOG


,id,name,date,high,low,close,adj_close,volume,daily_returns
0,1,GOOG,2022-11-28,97.830002,95.889999,96.250000,96.014206,19974500.0,NaN
1,2,GOOG,2022-11-29,96.389999,94.389999,95.440002,95.206192,20220000.0,-0.008416
2,3,GOOG,2022-11-30,101.449997,94.669998,101.449997,101.201469,39888100.0,0.062972
3,4,GOOG,2022-12-01,102.589996,100.669998,101.279999,101.031883,21771500.0,-0.001676
4,5,GOOG,2022-12-02,101.150002,99.169998,100.830002,100.582993,18821500.0,-0.004443
...,...,...,...,...,...,...,...,...,...
450,451,GOOG,2024-09-13,159.274994,156.110001,158.369995,158.369995,16733900.0,0.018195
451,452,GOOG,2024-09-16,159.240005,157.610001,158.990005,158.990005,14157600.0,0.003915
452,453,GOOG,2024-09-17,161.589996,159.410004,160.279999,160.279999,12064800.0,0.008114
453,454,GOOG,2024-09-18,161.630005,159.660004,160.809998,160.809998,16756500.0,0.003307


In [3]:
df["date"].iloc[0]

datetime.date(2022, 11, 28)

In [23]:
### Basic functions for daily returns

# Mapping intervals to approximate trading periods per year
intervals_per_year = {
    "1m": 525600,  # 60 * 24 * 365
    "2m": 262800,  # 30 * 24 * 365
    "5m": 105120,  # 12 * 60 * 365
    "15m": 35040,  # 4 * 60 * 365
    "30m": 17520,  # 2 * 60 * 365
    "60m": 8760,   # 60 * 365
    "90m": 5840,   # 60 * 365 / 1.5
    "1h": 8760,    # equivalent to 60m
    "1d": 252,     # trading days
    "5d": 52,      # trading weeks
    "1wk": 52,
    "1mo": 12,
    "3mo": 4
}


def valid_interval(interval):
    valid_intervals = ["1m", "2m", "5m", "15m", "30m", "60m", "90m", "1h", "1d", "5d", "1wk", "1mo", "3mo"]
    return interval in valid_intervals

# Arithmatic mean
def Arith_mean_r(df, annualise=False):
    mean_return = df["daily_returns"].mean()
    
    if not annualise:
        return mean_return
    else:
        interval = input("Interval data:")
        if not valid_interval(interval):
            print("Invalid interval.")
            return None
        
        if interval not in intervals_per_year:
            print("Interval not supported for annualization.")
            return None
        
        annualized_mean_return = mean_return * intervals_per_year[interval]
        return annualized_mean_return

# Geometric mean
def Geo_mean_r(df, annualise=False):
    add_one = df["daily_returns"] + 1
    cumul = add_one.prod()
    geometric_mean = cumul ** (1 / len(add_one)) - 1
    
    if not annualise:
        return geometric_mean
    else:
        interval = input("Interval data:")
        if not valid_interval(interval):
            print("Invalid interval.")
            return None

        if interval not in intervals_per_year:
            print("Interval not supported for annualization.")
            return None

        annualized_geometric_mean = (1 + geometric_mean) ** intervals_per_year[interval] - 1
        return annualized_geometric_mean

# Standard deviation
def std_r(df, annualise=False):
    if not annualise:
        return df["daily_returns"].std()
    else:
        interval = input("Interval data:")
        if not valid_interval(interval):
            print("Invalid interval.")
            return None
        
        # Check if the interval is valid
        if interval not in intervals_per_year:
            print("Interval not supported for annualization.")
            return None
        
        # Calculate standard deviation and annualize it
        std_dev = df["daily_returns"].std()
        annualized_std_dev = std_dev * (intervals_per_year[interval] ** 0.5)
        return annualized_std_dev
        

# Sharpe ratio with no rfr \raw input\
def sharpe(r,std):
    return r / std

sharpe(Arith_mean_r(df,annualise=True), std_r(df,annualise=True))

Interval data:1d
Interval data:1d


1.1398689711152736